In [1]:
from models.transformer_model import TransformerModel
from models.logistic_regression import LogRegressionModel
from dataloader.data_lr import DataModuleTask
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
from argparse import ArgumentParser
import numpy as np
import torch
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import plot_confusion_matrix, precision_recall_fscore_support, classification_report
from sklearn.metrics import f1_score, accuracy_score, classification_report  
from sklearn.decomposition import TruncatedSVD

/home/ryanchi/miniconda3/envs/sard/lib/python3.9/site-packages/google/auth/_default.py:70: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [3]:
# First time loading data?
dm = DataModuleTask(task='eol')
dm.setup(split=(0.8, 0.1))
torch.save(dm, 'saved/datamodule.pt')

# Otherwise:
# dm = torch.load('saved/datamodule.pt')

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/ryanchi/miniconda3/envs/sard/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3427, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-b507334e7fe6>", line 3, in <module>
    dm.setup(split=(0.8, 0.1))
  File "/home/ryanchi/miniconda3/envs/sard/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py", line 92, in wrapped_fn
    return fn(*args, **kwargs)
  File "/home/ryanchi/newest/prediction_tasks/dataloader/data_lr.py", line 213, in setup
    self.full_data = get_data(task=self.task)
  File "/home/ryanchi/newest/prediction_tasks/dataloader/data_lr.py", line 170, in __init__
    self.x, self.num_concepts = get_x(task)
  File "/home/ryanchi/newest/prediction_tasks/dataloader/data_lr.py", line 102, in get_x
    data = np.array(list(
  File "/home/ryanchi/newest/prediction_tasks/dataloader/data_lr.py", line 103, in <genexpr>
    [
  File "/home/ryanchi/newest/prediction_t

TypeError: object of type 'NoneType' has no len()

In [ ]:
def convert_dataloader_to_tensor(dataloader):
    data_iter = iter(dataloader)
    
    data_X_list = []
    data_y_list = []
    while True:
        try: 
            curr_example = next(data_iter)
            data_X_list.extend(curr_example[0])
            data_y_list.append(curr_example[1])
        except StopIteration:
            break

    data_X = torch.stack(data_X_list)
    data_y = torch.cat(data_y_list)
    
    return data_X, data_y

In [ ]:
# First time loading?
train_X, train_y = convert_dataloader_to_tensor(dm.train_dataloader())
val_X, val_y     = convert_dataloader_to_tensor(dm.val_dataloader())
test_X, test_y   = convert_dataloader_to_tensor(dm.test_dataloader())

torch.save(train_X, 'saved/train_X.pt')
torch.save(train_y, 'saved/train_y.pt')
torch.save(val_X, 'saved/val_X.pt')
torch.save(val_y, 'saved/val_y.pt')
torch.save(test_X, 'saved/test_X.pt')
torch.save(test_y, 'saved/test_y.pt')

In [ ]:
#Otherwise:
train_X = torch.load('saved/train_X.pt')
train_y = torch.load('saved/train_y.pt')
test_X = torch.load('saved/test_X.pt')
test_y = torch.load('saved/test_y.pt')
val_X = torch.load('saved/val_X.pt')
val_y = torch.load('saved/val_y.pt')

In [ ]:
print(train_X.shape)
print(train_y.shape)
print(val_X.shape)
print(val_y.shape)
print(test_X.shape)
print(test_y.shape)

In [ ]:
def regression_report(y_true, y_pred, estimator=""):
    """
        Helper function for error outputs.
        
        Parameters
        ----------
        y_true : numpy.1darray
            true labels for test data
        
        y_pred : numpy.1darray
            predicted labels for test data
        
        estimator : str
            name of estimator (for output purposes)
        
        Returns
        -------
        None
    """
    print(f"F1:  {f1_score(y_true, y_pred)}")
    print(f"F1 Macro:  {f1_score(y_true, y_pred, average='macro')}")

    print(f"Accuracy:  {accuracy_score(y_true, y_pred)}")

In [ ]:
lr = LogisticRegression(class_weight='balanced')
# lr = LogisticRegression()
lr.fit(train_X, train_y.view(-1))

In [ ]:
train_y_pred = lr.predict(train_X)
print(regression_report(train_y, train_y_pred))

In [ ]:
val_y_pred = lr.predict(val_X)
print(regression_report(val_y_pred, val_y))